In [57]:
%matplotlib widget
%load_ext autoreload
%autoreload 2


import matplotlib.pyplot as plt
import matplotlib.gridspec as GridSpec
import numpy as np
import os

from tqdm import tqdm, tqdm_notebook
import math

from  scipy.special import digamma as ψ
from mpmath import besseli as Iν
from scipy.constants import hbar, pi, h, e, k

kB = k
γ = 0.57721566
RQ = h/4/e**2
Φ0 = h/2/e

from JJformulas import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


function

In [40]:
os.getcwd()

'e:\\OneDrive - Rutgers University\\files_GitHub\\JJ arrays'

In [2]:
# def Ic (R, Δ = 2.1*kB):
#     return pi*Δ/2/e/R

# def EJ_AB (R, Δ = 2.1*kB):
    
#     Ic_ = Ic (R, Δ)
    
#     return Ic_*Φ0/2/pi/kB





    

# def EJ_star (EJ, R, T, C):
#     β = 1/T/kB
#     ωR = 1/R/C
#     ρ = R/RQ
    
    
#     return EJ*np.exp( -ρ*( ψ(1 + hbar*ωR*β/2/pi)+ γ) )
    

In [3]:
# def  I_IZ( Vb, EJ, R, T):
#     out = []

#     β = 1/T/kB
#     ρ = R/RQ
#     Z = 1j*β*e*Vb/pi/ρ

    
#     for z in Z:
       
#         try :
#             out = np.append(out, 2*e/hbar*EJ*kB * ( float((Iν(1-z, EJ/T) / Iν(-z, EJ/T)).imag) ))
#         except OverflowError:
#             print('¯\_(ツ)_/¯')
#             out = np.append(out, 0 )

#     return out

In [4]:
EJ_AB(150e3)

0.04517241304820373

In [5]:
Ic (150e3)

1.8950499176670637e-09

In [6]:
data = np.genfromtxt(os.getcwd()+ f'\IcvsEJ.dat')
IcIAB_exp = data[:, 2]
EJ0_exp = data[:, 0]
EJ0_exp

array([8.13031, 0.65602, 0.36831, 0.1087 , 0.04361])

In [60]:
R_list = np.logspace(2, 6, 101)

R_env = 1e5

EJs = EJ_star (EJ = EJ_AB(R_list), R = R_env, T = .03, C = 1.95e-15)

fig, ax1 = plt.subplots()
ax1.plot( EJ_AB(R_list) , EJs, label = r'C = 1.95 fF')

ax1.plot( EJ_AB(R_list) , EJ_AB(R_list), '--k', alpha = 0.5)
# ax.plot( EJ_AB(R_list) , EJ_AB(R_list)**2 / 5, '--k', alpha = 0.5)

ax1.plot(EJ0_exp, EJ0_exp * IcIAB_exp, 'o' , label = 'exp. data')


ax1.legend()

ax1.set_xscale('log')
ax1.set_yscale('log')

ax1.set_xlabel(r'$E_J, K$')
ax1.set_ylabel(r'$E_J^*, K$')

FigureCanvasNbAgg()

Text(0, 0.5, '$E_J^*, K$')

In [73]:
Rs = [1e3, 10e3, 20e3, 150e3 ]

R_env = 8e4
Tb = 0.05
Cj = 2.5e-15

n = len(Rs)

fig = plt.figure(figsize = (7,10))
gs = GridSpec.GridSpec(n, 1)



Vs = np.linspace(0, 1e-3, 101)


# Rs = [ 200e3 ]

for i, R in enumerate(Rs):
    
    ax = fig.add_subplot(gs[i] )#,sharex=ax)

    EJ_s = EJ_star (EJ = EJ_AB(R), R = R_env, T = Tb, C = Cj)
    Is = I_IZ( Vs, EJ = EJ_s, R = R_env, T = Tb) 


    ax.plot( (Vs - 1*Is*R_env)/1e-3, (Is)/1e-9, '-', label = 'Rn = {} kOhm, EJ* = {:1.3f} K'.format(R/1e3, EJ_s))

    ax.legend()    
    ax.set_xlabel(r'$V, mV$')
    ax.set_ylabel(r'$I, nA$')

# ax.set_xscale('log')
# ax.set_yscale('log')

plt.tight_layout()

FigureCanvasNbAgg()

In [11]:
# def find_Isw( R , T, C ):

#     Vs = np.linspace(0, .5e-3, 51)
    
#     EJ_s = EJ_star (EJ = EJ_AB(R), R = R, T = T, C = C)
    
    
#     Is = I_IZ( Vs, EJ = EJ_s, R = R, T = T) 

#     return np.max(Is)

In [17]:
# def find_R0( R , T, C ):

#     Vs = np.linspace(0, .1e-5, 51)
    
#     EJ_s = EJ_star (EJ = EJ_AB(R), R = R, T = T, C = C)
    
    
#     Is = I_IZ( Vs, EJ = EJ_s, R = R, T = T) 

    
    
#     return np.mean(np.diff(Vs - Is*R)/np.diff(Is)) +1

In [74]:
R_list = np.logspace(2, 5.5, 51)

Cj = 1.5e-15
Tb = 0.05

R_env = 8e4

EJs = EJ_star (EJ = EJ_AB(R_list), R = R_env, T = Tb, C = Cj)

fig, ax1 = plt.subplots()

Isw_maxs = []

for R in R_list:
    Isw_max = find_Isw( RN = R, R_env = R_env, T = Tb, C = Cj)
    Isw_maxs.append(Isw_max)

ax1.plot( EJ_AB(R_list) , np.array(Isw_maxs) / Ic(R_list), label = r'C = {:1.1f} fF'.format(Cj/1e-15))

# ax1.plot( EJ_AB(R_list) , EJ_AB(R_list), '--k', alpha = 0.5)
# ax.plot( EJ_AB(R_list) , EJ_AB(R_list)**2 / 5, '--k', alpha = 0.5)

ax1.plot(EJ0_exp, IcIAB_exp, 'o' , label = 'exp. data')


ax1.legend()

ax1.set_xscale('log')
ax1.set_yscale('log')

ax1.set_xlabel(r'$E_J, K$')
ax1.set_ylabel(r'$I_{sw}/I_{AB}$')

FigureCanvasNbAgg()

Text(0, 0.5, '$I_{sw}/I_{AB}$')

In [15]:
data = np.genfromtxt(os.getcwd()+ f'\R0vsEJ.dat')
R0_exp = data[:, 3]
EJ0_exp = data[:, 2]
EJ0_exp

array([16.26063,  1.31205,  0.73663,  0.2174 ,  0.08722])

In [68]:
R_list = np.logspace(2, 5.5, 51)

R_env = 2e5

Cj = 1.5e-15
Tb = 0.055

EJs = EJ_star (EJ = EJ_AB(R_list), R = R_list, T = Tb, C = Cj)

fig, ax1 = plt.subplots()

R0s = []

for R in R_list:
    R0 = find_R0( RN = R, R_env = R_env, T = Tb, C = Cj)
    R0s.append(R0)

ax1.plot( EJ_AB(R_list) , R0s, label = r'C = {:1.2f} fF'.format(Cj/1e-15))

# ax1.plot( EJ_AB(R_list) , EJ_AB(R_list), '--k', alpha = 0.5)
# ax.plot( EJ_AB(R_list) , EJ_AB(R_list)**2 / 5, '--k', alpha = 0.5)

ax1.plot(EJ0_exp, R0_exp, 'o' , label = 'exp. data')


ax1.legend()

ax1.set_xscale('log')
ax1.set_yscale('log')

ax1.set_xlabel(r'$E_J, K$')
ax1.set_ylabel(r'$R_0, k \Omega$')

FigureCanvasNbAgg()

Text(0, 0.5, '$R_0, k \\Omega$')